# 변수 분석 그래프 코드
변수 분석을 위해 결과 값을 기준으로

YN 분석
숫자형 데이터 분포 분석
카테고리 데이터 분포 분석 


In [ ]:
# AIDU Library Import

from aicentro.session import Session

from aicentro.framework.keras import Keras as AiduFrm

aidu_session = Session(verify=False)

aidu_framework = AiduFrm(session=aidu_session)

# 데이터 파일 경로

data_path = aidu_framework.config.data_dir

In [ ]:
#train_file 데이터 디렉토리
train_file_dir = data_path + "/episode1_train.csv"
train_ob_file_dir = data_path + "/episode1_train_ob.csv"

#train_file 한글로 변환한 디렉토리
train_han_file_dir = data_path + "/episode1_han_train.csv"
train_ob_han_file_dir = data_path + "/episode1_han_train_ob.csv"




In [ ]:
import pandas as pd
train_df = pd.read_csv(train_han_file_dir)
train_ob_df = pd.read_csv(train_ob_han_file_dir)

In [ ]:
train_raw_df.columns

## 변수 분석

In [ ]:
import numpy as np

def get_outlier(df=None, column=None, weight=1.5):
    '''
    outlier 추출하여 index로 리턴
    
    param 
    df: 추출하고자 하는 데이터 프레임 전체
    column : 이상치 확인 하려는 컬럼
    weight : 가중치
    
    '''
  # target 값과 상관관계가 높은 열을 우선적으로 진행
    quantile_25 = np.percentile(df[column].values, 25)
    quantile_75 = np.percentile(df[column].values, 75)

    IQR = quantile_75 - quantile_25
    IQR_weight = IQR*weight
  
    lowest = quantile_25 - IQR_weight
    highest = quantile_75 + IQR_weight
  
    outlier_idx = df[column][ (df[column] < lowest) | (df[column] > highest) ].index

    return outlier_idx


In [ ]:

cols =['유효이동전화가입건수','유효PSTN가입건수','유효인터넷가입건수','유효IPTV가입건수']

#이상치 값에 대해 YN분석
for index in cols:
    # 함수 사용해서 이상치 값 삭제
    oulier_idx = get_outlier(df=train_df, column=index, weight=1.5)
    oulier_idx

    outlier_idx_yn= train_df.loc[oulier_idx]['완납여부(Y=완납,N=미납)']
    # outlier_idx_yn.loc[outlier_idx_yn['완납여부(Y=완납,N=미납)'] ]
    print(index, "이상치 자체의 YN 비율은\n",outlier_idx_yn.value_counts(),"\n")

In [ ]:
#이상치 제거한 값에 대해 YN 분석

cols =['유효이동전화가입건수','유효PSTN가입건수','유효인터넷가입건수','유효IPTV가입건수']

for index in cols:
    # 함수 사용해서 이상치 값 삭제
    oulier_idx = get_outlier(df=train_df, column=index, weight=1.5)
    oulier_idx
    
    outlier_del_df = train_df.drop(index=oulier_idx, axis=0)['완납여부(Y=완납,N=미납)']
    print(index,"이상치 제거 후 데이터 NY 비율 \n",outlier_del_df.value_counts())


In [ ]:
#필요한 import 
!pip install seaborn
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# 1. Y/N 구성된 컬럼 분석 함수



In [ ]:

#원핫 인코딩

def yn_data_EDA(input_data_df, input_col, y_data_col):
    '''
    Y/N로 구성된 컬럼을 입력해주면 
    NN,YN,NY,YY 비율 출력
    
    파라미터:
    input_data_df: 분석하고 싶은 데이터 프레임 전체
    input_col: 분석하려는 컬럼 (Y/N)
    y_data_col : 결과값 (Y/N)
    '''
    plt.figure(figsize=(8, 5))
    

    print("@@@"+ input_col+"데이터 분석")
    confuse_df =input_data_df[[input_col,y_data_col]]
    
    confuse_df.rename(columns={input_col:'compare',y_data_col:'ans'},inplace=True)
 
    print("\n"+input_col+" 데이터 자체의 비율은:")
    print(confuse_df['compare'].value_counts(normalize=True)*100)
    
    #원핫인코딩
    confuse_df=pd.get_dummies(confuse_df)


    for index, com_col in enumerate(list(['compare_N','compare_Y'])):
        # 결과가 N일때
        N=confuse_df.loc[confuse_df[com_col]==1]['ans_N'].sum()
        # 결과가 Y일때
        Y=confuse_df.loc[confuse_df[com_col]==1]['ans_Y'].sum()
        
        height = [N,Y]
    
        bars = ( 'N',  'Y')
        plt.subplot(1,2,index+1)
        if index==0:
            plt.title(input_col+'_N일때')
        else:
            plt.title(input_col+'_Y일때')
            
        for num, x_label in enumerate(bars):
            plt.text(x_label,height[num],str(round(height[num]/(N+Y),3))+'%',
                    fontsize=12,
                    horizontalalignment='center',
                    verticalalignment='bottom')
        plt.ylim(0, N+Y)
        plt.bar(bars,height)

        
    plt.show()
    
    print("\n\n-------------------------------------------------------\n\n")

In [ ]:


#Yes No로 구성되어있는 컬럼만 추출
#원랜 맨 마지막 endswith(끝나는문자, 문자열의시작, 문자열의끝) 이걸로 찾아도 됨
#컬럼을 한글로 바꿔서  따로 제작

yn_col_list=[]


for col in train_df.columns:
    if train_df[col].dtype=='object':
        if train_df[col][0] in ['Y','N']:
            yn_col_list.append(col)


yn_col_list

 
        

In [ ]:
for i in yn_col_list:
    if i is not 'ans':
        yn_data_EDA(train_df,i,'ans')

# 2. 숫자형 데이터 그래프 그리기

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')
pd.options.display.float_format = '{:.5f}'.format

def int_data_EDA(input_data_df, input_col, ans_data_col):
    '''
    int 테이터 분석 함수
    1. 각각 구성이 및 비율
    2. Y데이터와 비교

    파라미터:
    input_data_df: 분석하고 싶은 데이터 프레임 전체
    input_col: 분석하려는 컬럼 (int)
    y_data_col : 결과값 (Y/N)
    '''

    
    #히스토그램 몇개로 쪼갤지 
    x_label_count = 25
    
    #최소 최대값 구한다.
    value_max_int = input_data_df[input_col].max()
    value_min_int = input_data_df[input_col].min()

    #한칸당 사이즈 크기 지정
    x_label_size_int = (value_max_int - value_min_int) / x_label_count

    #최소값부터 시작
    x_label_value_int = value_min_int

    
    #신규 데이터 프레임 만들어서 저장시킴
    compare_data_df = pd.DataFrame(columns=['Y_rate','N_rate','Y_count','N_count','X_col_mid'])

    #범위 안에 까지 확인
    while x_label_value_int <= value_max_int:
        
        # 범위에 해당하는 ans 갯수 확인
        ans_value_count = input_data_df.loc[
                    (input_data_df[input_col] > x_label_value_int)
                    & (input_data_df[input_col] < x_label_value_int + x_label_size_int),
                    ans_data_col].value_counts()

        # 원하는 범위에서  Y해당하는 값이 있으면 저장
        if 'Y' in list(ans_value_count.index) :
            y_count =  ans_value_count['Y']
        else : 
            y_count =  0
            
        # 원하는 범위에서  N해당하는 값이 있으면 저장    
        if 'N' in list(ans_value_count.index) :
            n_count =  ans_value_count['N']
        else : 
            n_count =  0
       
        #숫자대신 비율로 변경
        if y_count == 0 & n_count == 0:
            y_rate = 0
            n_rate = 0
        else :
            y_rate = y_count / (y_count+ n_count)
            n_rate = n_count / (y_count+ n_count)
            

        temp_dic={
            'Y_count' : y_count,
            'N_count' : n_count,
            'Y_rate' : y_rate,
            'N_rate' : n_rate,
            'X_col_mid': int(x_label_value_int + (x_label_size_int / 2))
        }
        
        #각 범위마다 Y개수, N개수, 범위 중간값을 데이터프레임으로 만듬
        compare_data_df = compare_data_df.append(temp_dic, ignore_index=True)
        
        #다음 범위로 넘어감
        x_label_value_int = x_label_value_int + x_label_size_int 
    
    
    plt.figure(figsize=(14, 5))
    plt.subplots_adjust(left=0.125, bottom=0.1,  right=0.9, top=0.9, wspace=0.2, hspace=0.35)
    
    #데이터 자체 히스토리 출력
    plt.subplot(1,3,1)
    plt.title(input_col+"  구성 히스토리 ")
    sns.distplot( a=input_data_df[input_col], hist=True, kde=False, rug=True)
   
    #Y비율 출력
    plt.subplot(1,3,2)
    plt.title(input_col+"값 별  "+ans_data_col+'_Y 비율')
    plt.bar(range(compare_data_df.shape[0]), compare_data_df['Y_rate'])
    plt.xticks(range(compare_data_df.shape[0]), compare_data_df['X_col_mid'],rotation=60)
    plt.ylim(0,1)
    
    #N비율 출력
    plt.subplot(1,3,3)
    plt.title(input_col+"값 별  "+ ans_data_col+'_N 비율')
    plt.bar(range(compare_data_df.shape[0]), compare_data_df['N_rate'])
    plt.xticks(range(compare_data_df.shape[0]), compare_data_df['X_col_mid'],rotation=60)
    plt.ylim(0,1)
    plt.show()
    
       
    print(input_data_df[input_col].describe(),"\n")
    print(compare_data_df)
    print("\n\n---------------------------------\n\n")
    





In [ ]:

train_df

int_col_list=[]


for col in train_df.columns:
    if train_df[col].dtype != 'object':
        int_data_EDA(train_df, col, 'ans')
        

# 3. 카테고리형 데이터 그래프 출력

In [ ]:
def category_data_EDA(input_data_df, input_col, ans_data_col):
    '''
    카테고리 분석 함수
    1. 각각 구성이 및 비율
    2. Y데이터와 비교

    파라미터:
    input_data_df: 분석하고 싶은 데이터 프레임 전체
    input_col: 분석하려는 컬럼 (카테고리)
    y_data_col : 결과값 (Y/N)
    '''
    
    
    

    #신규 데이터 프레임 만들어서 저장시킴
    compare_data_df = pd.DataFrame(columns=['Y_rate','N_rate','Y_count','N_count','X_col'])

    #범위 안에 까지 확인
    for item in list(input_data_df[input_col].unique()):

        
        # 범위에 해당하는 ans 갯수 확인
        ans_value_count = input_data_df.loc[
                    (input_data_df[input_col] == item),
                    ans_data_col].value_counts()

        # 원하는 범위에서  Y해당하는 값이 있으면 저장
        if 'Y' in list(ans_value_count.index) :
            y_count =  ans_value_count['Y']
        else : 
            y_count =  0
            
        # 원하는 범위에서  N해당하는 값이 있으면 저장    
        if 'N' in list(ans_value_count.index) :
            n_count =  ans_value_count['N']
        else : 
            n_count =  0
       
        #숫자대신 비율로 변경
        if y_count == 0 & n_count == 0:
            y_rate = 0
            n_rate = 0
        else :
            y_rate = y_count / (y_count+ n_count)
            n_rate = n_count / (y_count+ n_count)
            

        temp_dic={
            'Y_count' : y_count,
            'N_count' : n_count,
            'Y_rate' : y_rate,
            'N_rate' : n_rate,
            'X_col': item
        }
        
        #각 범위마다 Y개수, N개수, 범위 중간값을 데이터프레임으로 만듬
        compare_data_df = compare_data_df.append(temp_dic, ignore_index=True)
        

    
    
    plt.figure(figsize=(14, 5))
    plt.subplots_adjust(left=0.125, bottom=0.1,  right=0.9, top=0.9, wspace=0.2, hspace=0.35)
    
    #데이터 자체 히스토리 출력
    plt.subplot(1,3,1)
    plt.title(input_col+"  구성 히스토리 ")
    plt.hist( input_data_df[input_col])
    plt.xticks(rotation=60)
   
    #Y비율 출력
    plt.subplot(1,3,2)
    plt.title(input_col+"값 별  "+ans_data_col+'_Y 비율')
    plt.bar(range(compare_data_df.shape[0]), compare_data_df['Y_rate'])
    plt.xticks(range(compare_data_df.shape[0]), compare_data_df['X_col'],rotation=60)
    plt.ylim(0,1)
    
    #N비율 출력
    plt.subplot(1,3,3)
    plt.title(input_col+"값 별  "+ ans_data_col+'_N 비율')
    plt.bar(range(compare_data_df.shape[0]), compare_data_df['N_rate'])
    plt.xticks(range(compare_data_df.shape[0]), compare_data_df['X_col'],rotation=60)
    plt.ylim(0,1)
    plt.show()
    
       
    print(input_data_df[input_col].describe(),"\n")
    print(compare_data_df)
    print("\n\n---------------------------------\n\n")
    


In [ ]:
#카테고리로 구성된 컬럼 추출
train_df

train_cat_col_list=[]




for col in train_df.columns:
    if train_df[col].dtype == 'object':
        if len(train_df[col].unique()) != 2:
             if len(train_df[col].unique())< 30:
                train_cat_col_list.append(col)
            
                category_data_EDA(train_df, col, 'ans')
            
print(train_cat_col_list)
